# BigQueryVectorSearch
> **BigQueryVectorSearch**:
BigQuery vector search lets you use GoogleSQL to do semantic
search, using vector indexes for fast but approximate results,
or using brute force for exact results.


This tutorial illustrates how to work with an end-to-end data
and embedding management system in LangChain, and provide scalable
semantic search in BigQuery.

## Getting started


### Install the library

In [ ]:
! pip install \
    google-cloud-aiplatform \
    langchain==0.0.316 \
    google-cloud-bigquery \
    pydantic==1.10.8 \
    typing-inspect==0.8.0 \
    typing_extensions==4.5.0 \
    pandas \
    openai==0.28.1 \
    tiktoken \
    datasets \
    google-api-python-client \
    pypdf \
    faiss-cpu \
    transformers \
    config \
    --upgrade \
    --user

**Colab only:** Uncomment the following cell to restart the
kernel or use the button to restart the kernel. For Vertex
AI Workbench you can restart the terminal using the button
on top.

In [ ]:
# # Automatically restart kernel after installs so that your
# # environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

#### Set your project ID

If you don't know your project ID, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page:
[Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [ ]:
PROJECT_ID = ""  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Set the region

You can also change the `REGION` variable used by BigQuery.
Learn more about 
[BigQuery regions](https://cloud.google.com/bigquery/docs/locations\
#supported_locations).

In [ ]:
REGION = "US"  # @param {type: "string"}

### Authenticating your notebook environment

- If you are using **Colab** to run this notebook, uncomment the cell
below and continue.
- If you are using **Vertex AI Workbench**, check out the setup
instructions
[here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
from google.colab import auth as google_auth

google_auth.authenticate_user()

## Demo: BigQueryVectorSearch

In [ ]:
from langchain.vectorstores import BigQueryVectorSearch

### Create an embedding in VectorStore

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
import os
import getpass

# We want to use OpenAIEmbeddings so we have to get the OpenAI API Key.
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [ ]:
embedding = OpenAIEmbeddings()

### Initialize VectorStore from a list of strings + embeddings

In [ ]:
all_texts = [
    "Apples and oranges",
    "Cars and airplanes",
    "Pineapple",
    "Train",
    "Banana"
]

store = BigQueryVectorSearch(
    embedding,
    project_id=PROJECT_ID,
    dataset_name="<your_dataset>",
    table_name="<your_table>",
    location=REGION)

### Intialize VectorStore with existing dataset with embedding columns in BQ

In [ ]:
from langchain.vectorstores.utils import DistanceStrategy

DEFAULT_DISTANCE_STRATEGY = DistanceStrategy.EUCLIDEAN_DISTANCE

bq_vector_search = BigQueryVectorSearch(
    project_id=PROJECT_ID,
    dataset_name="your_dataset",
    table_name="<your_table>",
    # Column {content_field} must be of STRING type
    content_field="<your_content>",
    # Column {text_embedding_field} must be of ARRAY<FLOAT64> type
    text_embedding_field="<your_embedding>",
    embedding=embedding,
    distance_strategy=DEFAULT_DISTANCE_STRATEGY,
    location=REGION)

### Add texts

In [ ]:
all_texts = [
    "Apples and oranges",
    "Cars and airplanes",
    "Pineapple",
    "Train",
    "Banana"
]

store.add_texts(all_texts)

### Search for documents

In [ ]:
query = "I'd like a fruit."
docs = store.similarity_search(query)

### Search for documents by vector

In [ ]:
query_vector = embedding.embed_query(query)
docs = store.similarity_search_by_vector(query_vector,k=2)

### Search for documents with metadata filter

In [ ]:
docs = store.similarity_search_by_vector(
    query_vector,
    filter={"float_t": 1.23}
)